In [ ]:
from pyspark.sql.functions import *

# Define some constants
RATING_MIN = 0.5
RATING_MAX = 5.0
RATING_RANGE = RATING_MAX - RATING_MIN

# Enable crossjoins
print("Configuring Spark...")
spark.conf.set("spark.sql.crossJoin.enabled", True)

# Define some helper functions
def readCSV(fname, removeHeader=False, separator=','):
    print("Loading file ", fname, "...")
    rdd = sc.textFile(fname)
    if removeHeader:
        firstline = rdd.first()
        rdd = rdd.filter(lambda x: x != firstline)
    return rdd.map(lambda x: x.split(separator))

# Load the movies and ratings database
movies = readCSV("./movies.csv", removeHeader=True)
ratings = readCSV("./ratings_train.csv", removeHeader=True)

# Print the first entries to debug whether the data is loaded correctly
print("First movie:", movies.first())
print("First rating:", ratings.first())

# Parse the movie genres
# [id, name, genres[]]
movies = movies.map(lambda x: [x[0], x[1], x[2].split('|')])
print("First movie, processed:", movies.first())

# Parse the rating data
# [user_id, movie_id, rating, timestamp]
ratings = ratings.map(lambda x: x[0].split('::'))
print("First rating, processed:", ratings.first())

print("Caching ratings...")
ratings = ratings.cache()

# Select the user to suggest movies for
client = ratings.first();
client_id = client[0];
#print("Determining movie suggestions for user", client[0], "...")
#
#def addToSet(input_set, value):
#    input_set.add(value)
#    return input_set
#
## Group all ratings by their user keys
#user_ratings = ratings.map(lambda x: (x[0], tuple(x[1:])))\
#                   .aggregateByKey(\
#                       set(), # initial value for an accumulator \
#                       addToSet, # function to add a value to an accumulator \
#                       lambda r1, r2: r1.union(r2) # function to merge two accumulators \
#                   )
#
## Get the ratings for the selected client
#client_ratings = user_ratings.lookup(client_id)
#print("Client ratings:", client_ratings)
#
## The selected client must not be in the list of user ratings
#user_ratings = user_ratings.filter(lambda x: x[0] != client_id)
#
#print("Caching user ratings...")
#user_ratings = user_ratings.cache()

# Create a data frame with all the ratings
ratings_df = spark.createDataFrame(ratings, ['user_id', 'movie_id', 'rating', 'timestamp'])
client_ratings_df = ratings_df.filter(ratings_df.user_id == client_id).alias("client")
user_ratings_df = ratings_df.filter(ratings_df.user_id != client_id).alias("other")
movies_df = spark.createDataFrame(movies, ['id', 'name', 'genres'])\
    .select("id", "name").alias("movies")

#####################################
print("user_ratings_df:")
user_ratings_df.show(5)

# Join the movies watched by the client and the other user
aaa = user_ratings_df.join(client_ratings_df, on = "movie_id", how = "inner")

# Determine the rating distance for each user/movie pair, and normalize it
aaa = aaa.select(\
                 col("client.user_id").alias("sim_client_user_id"),\
                 col("other.user_id").alias("sim_other_user_id"),\
                 "movie_id",\
                 (abs(col("client.rating") - col("other.rating")) - RATING_RANGE / 2.0).alias("rating_dist_norm")\
                )

#####################################
print("aaa:")
aaa.show(5)

# For each client, calculate the similarity to the other users
users_similarity = aaa.groupBy("sim_client_user_id", "sim_other_user_id")\
    .agg(sum("rating_dist_norm").alias("similarity")).alias("similarity")

## Create a list of users
print("Creating list of users...")
user_list_df = ratings_df.select(col("user_id").alias("list_user_id")).distinct()
# TODO: Do not limit to 1 in production!!!
user_list_df = user_list_df.limit(1)

#####################################
print("user_list_df:")
user_list_df.show(1)

suggestions = user_list_df.join(\
                                movies_df.select(\
                                                 col("id").alias("sug_movie_id")\
                                                ),\
                                on = col("sug_movie_id") != True,\
                                how = "inner")

#####################################
print("suggestions:")
suggestions.show(5)

suggestions = suggestions.join(user_ratings_df,\
                               on = [col("sug_movie_id") == col("movie_id"),\
                                     col("list_user_id") != col("user_id")],\
                               how = "inner")

suggestions = suggestions.join(\
                               users_similarity,\
                               on = [col("list_user_id") == col("sim_client_user_id"),\
                                     col("user_id") == col("sim_other_user_id")],\
                              how = "inner")

# TODO: Filter movies that have been watched already here



#unwatched.filter(client_ratings_df\
#                 .select(client_ratings_df.movie_id)\
#                 .where(client_ratings_df.movie_id == unwatched.id)\
#                 .limit(1)\
#                 .count() == 0)


suggestions.show(1000)


#unwatched_df = client_ratings_df.join(movies_df, on = client_ratings_df.movie_id != movies_df.id, how = "left")
#unwatched_df.show(10000)

#def abc_test(user):
#    print("For user", user.user_id)
#    #user_movies = movies_df.filter(\
#    #                               ratings_df.where(col("user_id") == user.user_id)\
#    #                                   .filter(movies_df.id == ratings_df.movie_id)\
#    #                                   .limit(1)\
#    #                                   .count() == 0
#    #                              )
#    #
#    #user_movies.show(1000)
#    pass
#
#print("Number of users:", user_list_df.count())
#user_list_df.foreach(abc_test)

#unwatched_df = client_ratings_df.join(movies_df, on = client_ratings_df.movie_id == movies_df.id, how = "outer")
#unwatched_df = client_ratings_df.crossJoin(movies_df)
    
#unwatched_df.sort(asc("user_id"), asc("movie_id")).show(10000)
#unwatched_df.show(10000)

#aaa.sort(desc('rating_dist_norm')).show(100)
#aaa.show(100)

# Aggregate o
# user_ratings.mapValues(lambda x: )

# TEST: Calculate rating averages
# print("Calculating ratings average...")
# user_ratings = ratings.map(lambda x: float(x[2]))
# print("Avg rating:", user_ratings.sum()/user_ratings.count())

print("DONE")

Configuring Spark...
Loading file  ./movies.csv ...


Py4JJavaError: An error occurred while calling o90.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/movies.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:199)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:248)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:246)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:246)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:248)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:246)
	at scala.Option.getOrElse(Option.scala:121